In [ ]:
import lightkurve as lk
import numpy as np
from csv import reader
import joblib
import os
import sys
import warnings
import time
import multiprocessing 

data_id = []
data_class = []
data_period = []
data_epoch = []
data_duration = []

print("Reading csv")

with open('Dataset_cleaned.csv') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader)
    for row in csv_reader:
        data_id.append(row[0])
        data_class.append(row[1])
        data_period.append(float(row[2]))
        data_epoch.append(float(row[3]))
        data_duration.append(float(row[4]))


def one():

    print("1-started")

    try:
        array = joblib.load('Dataset_test_bls_alt_4.joblib')
        joblib.dump(array, 'Dataset1_test_bls.joblib')

    except:
        array = joblib.load('Dataset1_test_bls_alt_4.joblib')
        print("Dataset.joblib corrupted,using Dataset1.joblib")
        joblib.dump(array, 'Dataset_test_bls_alt_4.joblib')

    print("1-loaded")


    index = 6000 + len(array[0])

    Dataset_global = array[0]
    Dataset_local = array[1]

    dump_i = 0

    
    print("1-Creating dataset")

    for x in range(index,len(data_id)):

        data_global = [data_id[x],data_class[x]]
        data_local = [data_id[x],data_class[x]]
        try:
            lcs = lk.search_lightcurve(data_id[x],author='Kepler',cadence='long').download_all()
            lc = lcs.stitch()
            lc_clean = lc.remove_outliers(sigma=20, sigma_upper=4)
            temp_fold = lc_clean.fold(data_period[x], epoch_time=data_epoch[x])
            fractional_duration = (data_duration[x] / 24.0) / data_period[x]
            phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
            transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
            lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
            lc_fold = lc_flat.fold(data_period[x], epoch_time=data_epoch[x])
            lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
            lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1

            phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
            lc_zoom = lc_fold[phase_mask]
            lc_local = lc_zoom.bin(time_bin_size=0.0005).normalize() - 1
            lc_local = (lc_local / np.abs(np.nanmin(lc_local.flux)) ) * 2.0 + 1


            for x in range(0,len(lc_global.flux)):
                try:
                    data_global.append(float(lc_global.flux[x].value))
                    data_local.append(float(lc_local.flux[x].value))
                except:
                    pass

            Dataset_global.append(data_global)
            Dataset_local.append(data_local)
            index = index + 1
            print(index)
            dump_i = dump_i + 1
            if index == 6998 :
                print("1-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_4.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_4.joblib')
                print("1-Dump Complete")
                break
            if dump_i == 100 :
                dump_i = 0
                print("1-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_4.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_4.joblib')
                print("1-Dump Complete")

        except :
            print("1-Download/Stitch error! #")
            index = index + 1 
            
def two():
    
    print("2-started")

    try:
        array = joblib.load('Dataset_test_bls_alt_5.joblib')
        joblib.dump(array, 'Dataset1_test_bls_alt_5.joblib')

    except:
        array = joblib.load('Dataset1_test_bls_alt_5.joblib')
        print("Dataset.joblib corrupted,using Dataset1.joblib")
        joblib.dump(array, 'Dataset_test_bls_alt_5.joblib')

    print("2-loaded")


    index = 7000 + len(array[0])

    Dataset_global = array[0]
    Dataset_local = array[1]

    dump_i = 0

    print("2-Creating dataset")

    for x in range(index,len(data_id)):

        data_global = [data_id[x],data_class[x]]
        data_local = [data_id[x],data_class[x]]
        try:
            lcs = lk.search_lightcurve(data_id[x],author='Kepler',cadence='long').download_all()
            lc = lcs.stitch()
            lc_clean = lc.remove_outliers(sigma=20, sigma_upper=4)
            temp_fold = lc_clean.fold(data_period[x], epoch_time=data_epoch[x])
            fractional_duration = (data_duration[x] / 24.0) / data_period[x]
            phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
            transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
            lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
            lc_fold = lc_flat.fold(data_period[x], epoch_time=data_epoch[x])
            lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
            lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1

            phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
            lc_zoom = lc_fold[phase_mask]
            lc_local = lc_zoom.bin(time_bin_size=0.0005).normalize() - 1
            lc_local = (lc_local / np.abs(np.nanmin(lc_local.flux)) ) * 2.0 + 1


            for x in range(0,len(lc_global.flux)):
                try:
                    data_global.append(float(lc_global.flux[x].value))
                    data_local.append(float(lc_local.flux[x].value))
                except:
                    pass

            Dataset_global.append(data_global)
            Dataset_local.append(data_local)
            index = index + 1
            print(index)
            dump_i = dump_i + 1
            if index == 7998 :
                print("2-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_5.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_5.joblib')
                print("2-Dump Complete")
                break
            if dump_i == 100 :
                dump_i = 0
                print("2-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_5.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_5.joblib')
                print("2-Dump Complete")

        except :
            print("2-Download/Stitch error! #")
            index = index + 1 

def three():
    
    print("3-started")

    try:
        array = joblib.load('Dataset_test_bls_alt_6.joblib')
        joblib.dump(array, 'Dataset1_test_bls_alt_6.joblib')

    except:
        array = joblib.load('Dataset1_test_bls_alt_6.joblib')
        print("Dataset.joblib corrupted,using Dataset1.joblib")
        joblib.dump(array, 'Dataset_test_bls_alt_6.joblib')

    print("3-loaded")


    index = 8000 + len(array[0])

    Dataset_global = array[0]
    Dataset_local = array[1]

    dump_i = 0


    print("3-Creating dataset")

    for x in range(index,len(data_id)):

        data_global = [data_id[x],data_class[x]]
        data_local = [data_id[x],data_class[x]]
        try:
            lcs = lk.search_lightcurve(data_id[x],author='Kepler',cadence='long').download_all()
            lc = lcs.stitch()
            lc_clean = lc.remove_outliers(sigma=20, sigma_upper=4)
            temp_fold = lc_clean.fold(data_period[x], epoch_time=data_epoch[x])
            fractional_duration = (data_duration[x] / 24.0) / data_period[x]
            phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
            transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
            lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
            lc_fold = lc_flat.fold(data_period[x], epoch_time=data_epoch[x])
            lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
            lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1

            phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
            lc_zoom = lc_fold[phase_mask]
            lc_local = lc_zoom.bin(time_bin_size=0.0005).normalize() - 1
            lc_local = (lc_local / np.abs(np.nanmin(lc_local.flux)) ) * 2.0 + 1


            for x in range(0,len(lc_global.flux)):
                try:
                    data_global.append(float(lc_global.flux[x].value))
                    data_local.append(float(lc_local.flux[x].value))
                except:
                    pass

            Dataset_global.append(data_global)
            Dataset_local.append(data_local)
            index = index + 1
            print(index)
            dump_i = dump_i + 1
            if index == 8998 :
                print("3-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_6.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_6.joblib')
                print("3-Dump Complete")
                break
            if dump_i == 100 :
                dump_i = 0
                print("3-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_6.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_6.joblib')
                print("3-Dump Complete")

        except :
            print("3-Download/Stitch error! #")
            index = index + 1 


def four():

    print("4-started")

    try:
        array = joblib.load('Dataset_test_bls_alt_7.joblib')
        joblib.dump(array, 'Dataset1_test_bls_alt_7.joblib')

    except:
        array = joblib.load('Dataset1_test_bls_alt_7.joblib')
        print("Dataset.joblib corrupted,using Dataset1.joblib")
        joblib.dump(array, 'Dataset_test_bls_alt_7.joblib')

    print("4-loaded")


    index = 9000 + len(array[0])

    Dataset_global = array[0]
    Dataset_local = array[1]

    dump_i = 0


    print("4-Creating dataset")

    for x in range(index,len(data_id)):

        data_global = [data_id[x],data_class[x]]
        data_local = [data_id[x],data_class[x]]
        try:
            lcs = lk.search_lightcurve(data_id[x],author='Kepler',cadence='long').download_all()
            lc = lcs.stitch()
            lc_clean = lc.remove_outliers(sigma=20, sigma_upper=4)
            temp_fold = lc_clean.fold(data_period[x], epoch_time=data_epoch[x])
            fractional_duration = (data_duration[x] / 24.0) / data_period[x]
            phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
            transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
            lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
            lc_fold = lc_flat.fold(data_period[x], epoch_time=data_epoch[x])
            lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
            lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1

            phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
            lc_zoom = lc_fold[phase_mask]
            lc_local = lc_zoom.bin(time_bin_size=0.0005).normalize() - 1
            lc_local = (lc_local / np.abs(np.nanmin(lc_local.flux)) ) * 2.0 + 1


            for x in range(0,len(lc_global.flux)):
                try:
                    data_global.append(float(lc_global.flux[x].value))
                    data_local.append(float(lc_local.flux[x].value))
                except:
                    pass

            Dataset_global.append(data_global)
            Dataset_local.append(data_local)
            index = index + 1
            print(index)
            dump_i = dump_i + 1
            if index == 9998 :
                print("4-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_7.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_7.joblib')
                print("4-Dump Complete")
                break
            if dump_i == 100 :
                dump_i = 0
                print("4-Dumping")
                joblib.dump([Dataset_global,Dataset_local], 'Dataset_test_bls_alt_7.joblib')
                joblib.dump([Dataset_global,Dataset_local], 'Dataset1_test_bls_alt_7.joblib')
                print("4-Dump Complete")

        except :
            print("4-Download/Stitch error! #")
            index = index + 1 

            

if __name__ == '__main__':
    starttime = time.time()
    processes = []

    p = multiprocessing.Process(target=one)
    processes.append(p)
    p.start()
    
    q = multiprocessing.Process(target=two)
    processes.append(q)
    q.start()
    
    r = multiprocessing.Process(target=three)
    processes.append(r)
    r.start()
    
    s = multiprocessing.Process(target=four)
    processes.append(s)
    s.start()
        
    for process in processes:
        process.join()


Reading csv
1-started
2-started
3-started
4-started
3-loaded
3-Creating dataset
1-loaded
1-Creating dataset
4-loaded
4-Creating dataset


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


4-Download/Stitch error! #
2-loaded
2-Creating dataset
8601
8602
7301
7302
8603
7303
8604
7304
8605
8606
9202
8607
9203
8608
8609
8610
1-Download/Stitch error! #
8611
6202
8612
2-Download/Stitch error! #
7306
8613
7307
8614
7308
8615
7309


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


3-Download/Stitch error! #
7310
8617
4-Download/Stitch error! #


...

3-Download/Stitch error! #


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


3-Download/Stitch error! #
7322
9208
7323
8640
7324
8641
8642
7325
6209
8643
7326
8644
6210
4-Download/Stitch error! #
7327


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


4-Download/Stitch error! #
7328
7329
